In [9]:
import pandas as pd
import numpy as np
import streamlit as st
import matplotlib.pyplot as plt

#reading csv:s to dataframes
df_price = pd.read_csv("Electricity_price_2015-2025.csv", sep =";", decimal = ",")
df_cons = pd.read_csv("Electricity_consumption_2015-2025.csv")

df_price = df_price.dropna()
df_cons = df_cons.dropna()

#changing times to pd.datetimes
df_price['timestamp'] = pd.to_datetime(df_price['timestamp'])
df_cons['time'] = pd.to_datetime(df_cons['time'])
df_cons.rename(columns = {'time': 'timestamp'}, inplace = True)

df = pd.merge(df_price, df_cons, on = 'timestamp')

#Forming hourly rate from price and Kwh per hour
df['hourly_rate'] = df['Price'] * df['kWh']

#Forming date and month columns so we can aggregate values later
df['day'] = pd.to_datetime(df['timestamp']).dt.date
df['month'] = df['timestamp'].dt.to_period('M')

#Forming new dfs to show hourly, daily and monthly values
df_hourly = df.groupby(['timestamp'], as_index = False).agg({'Price' : 'mean', 'kWh' : 'sum', 'Temperature' : 'mean'})
df_daily = df.groupby(['day'], as_index= False).agg({'Price' : 'mean', 'kWh' : 'sum', 'Temperature' : 'mean'})
df_monthly = df.groupby(['month'], as_index= False).agg({'Price' : 'mean', 'kWh' : 'sum', 'Temperature' : 'mean'})


#Renaming columns to match hourly, daily and monthly data
df_hourly.rename(columns = {'Price' : 'mean_price', 'kWh' : 'total_kwh', 'Temperature' : 'mean_temperature', 'timestamp': 'hour'}, inplace = True)
df_daily.rename(columns = {'Price' : 'mean_price', 'kWh' : 'total_kwh', 'Temperature' : 'mean_temperature'}, inplace = True)
df_monthly.rename(columns = {'Price' : 'mean_price', 'kWh' : 'total_kwh', 'Temperature' : 'mean_temperature'}, inplace = True)

#Forming total_consumption column to all dataframes
df_hourly['total_consumption'] = df_hourly['mean_price'] * df_hourly['total_kwh']

df_daily['total_consumption'] = df_daily['mean_price'] * df_daily['total_kwh']
df_monthly['total_consumption'] = df_monthly['mean_price'] * df_daily['total_kwh']




#print(df_hourly)
print(df_hourly.dtypes)
#print(df_monthly)
df_hourly

hour                 datetime64[ns]
mean_price                  float64
total_kwh                   float64
mean_temperature            float64
total_consumption           float64
dtype: object


,hour,mean_price,total_kwh,mean_temperature,total_consumption
0,2015-01-01 00:00:00,3.47,3.670,1.2,12.73490
1,2015-01-01 01:00:00,3.40,4.148,1.5,14.10320
2,2015-01-01 02:00:00,2.90,4.342,1.6,12.59180
3,2015-01-01 03:00:00,2.40,2.095,2.8,5.02800
4,2015-01-01 04:00:00,2.19,0.985,3.4,2.15715
...,...,...,...,...,...
93337,2025-09-02 19:00:00,15.89,0.335,16.3,5.32315
93338,2025-09-02 20:00:00,13.08,0.369,14.4,4.82652
93339,2025-09-02 21:00:00,7.94,0.598,11.7,4.74812
93340,2025-09-02 22:00:00,5.58,2.889,11.2,16.12062


In [4]:
fig, ax = plt.subplots()
ax.bar(df_monthly['month'], df_monthly['total_kwh'])
plt.show()

TypeError: the dtypes of parameters x (object) and width (float64) are incompatible